In [1]:
import tensorflow as tf
import numpy as np

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X = np.load(r'D:\learning_data\time_translation\x.npy')
Y = np.load(r'D:\learning_data\time_translation\y.npy')
X_test = X[9000:10000]
Y_test = Y[9000:10000]
Y = Y[0:9000]
X = X[0:9000]
now_batch_size = tf.placeholder(shape=[],dtype=tf.int32,name="batch_size")
print(X_test.shape)

(1000, 30, 37)


In [3]:
print(X.shape)
print(Y.shape)

(9000, 30, 37)
(9000, 10, 11)


In [4]:
human_vocab = {' ': 0, '.': 1, '/': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11,
                   '9': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22,
                   'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33,
                   'y': 34, '<unk>': 35, '<pad>': 36}
machine_vocab = {'-': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10}
inv_machine_vocab = {0: '-', 1: '0', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '6', 8: '7', 9: '8', 10: '9'}

In [6]:
n_a = 32
n_s = 64
Tx = 30
Ty = 10
len_human_vocabulary = len(human_vocab)
print(len_human_vocabulary)
len_machine_vocabulary = len(machine_vocab)
print(len_machine_vocabulary)
minibatch_size = 64

37
11


In [7]:
x_placeholder = tf.placeholder(dtype=tf.float32,shape=[None,Tx,len_human_vocabulary],name='x_placeholder')
y_placeholder = tf.placeholder(dtype=tf.float32,shape=[None,Ty,len_machine_vocabulary],name='y_placeholder')

In [8]:
lstm_cell_pre1 = tf.contrib.rnn.BasicLSTMCell(n_a)
lstm_cell_pre2 = tf.contrib.rnn.BasicLSTMCell(n_a)
a,_ = tf.nn.bidirectional_dynamic_rnn(lstm_cell_pre1,lstm_cell_pre2,x_placeholder,dtype=tf.float32)
a = tf.concat([a[0],a[1]],-1)

In [9]:
lstm_cell_post = tf.contrib.rnn.BasicLSTMCell(n_s,reuse=tf.AUTO_REUSE)

In [10]:
cell_state_0_par = tf.Variable(tf.random_normal([1,n_s]))
hidden_state_0_par = tf.Variable(tf.random_normal([1,n_s]))
# cell_state_0_holder = tf.placeholder(shape=[None,n_s],dtype=tf.float32)
# hidden_state_0_holder = tf.placeholder(shape=[None,n_s],dtype=tf.float32)
# cell_state = tf.tile(cell_state_0_holder,[minibatch_size,1])
# hidden_state  = tf.tile(hidden_state_0_holder,[minibatch_size,1])
cell_state_0 = tf.tile(cell_state_0_par,[now_batch_size,1])
hidden_state_0 = tf.tile(hidden_state_0_par,[now_batch_size,1])
print(cell_state_0.shape)
# b = tf. 
# c = lstm_cell_post.call(b,LSTM_state)

(?, 64)


In [11]:
print(a.shape)
# cd = tf.layers.dense(inputs=a,units=10,reuse=tf.AUTO_REUSE)

(?, 30, 64)


In [12]:
def one_step_attention(a,s_pre):
    s_pre = tf.expand_dims(s_pre,axis=1)
#     print("after exp:",s_pre.shape)
    s_pre = tf.tile(s_pre,[1,Tx,1])
#     print("after repeating:",s_pre.shape)
    hidden = tf.layers.dense(inputs=tf.concat([a,s_pre],-1),units=10,reuse=tf.AUTO_REUSE,name='attention_dense_1')
    energies = tf.layers.dense(inputs = hidden,units=1,reuse=tf.AUTO_REUSE,name='attention_dense_2')
    energies = tf.transpose(energies,[0,2,1])
    e = tf.nn.softmax(energies,axis=-1)
#     print(e.shape)
    context = tf.matmul(e,a)
#     context = tf.multiply(a,e)
#     print(context.shape)
    context = tf.reduce_sum(context,-2)
#     print(context.shape)
    return context

In [13]:
# ta = tf.placeholder(shape=[None,64],dtype=tf.float32)
# ts = tf.placeholder(shape=[None,64],dtype=tf.float32)
# lxn = tf.nn.rnn_cell.BasicLSTMCell(num_units=64)
# lxn.__call__(ta,[cell_state_0,hidden_state_0_])
# # one_step_attention(a,hidden_state_0_holder)

In [14]:
# lxn1 = tf.placeholder(shape=(100,64),dtype=tf.float32,name='sss')
# # lxn2 = tf.placeholder(shape=(None,25),dtype=tf.float32)
# test1 = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_s,reuse=tf.AUTO_REUSE)
# state111 = tf.placeholder(shape=(100,n_s),dtype=tf.float32)
# # test1.__call__(inputs=lxn1,state=LSTM_state)
# print(test1.__call__(inputs=lxn1,state=[state111,state111]))
# test1.call(inputs=lxn1,state=[state111,state111])
context = one_step_attention(a,h)
print(context.shape)

NameError: name 'h' is not defined

In [15]:
h = hidden_state_0
c = cell_state_0
print(h.shape)
print(c.shape)
predict = tf.placeholder
outputs = []
for t in range(Ty):
    context = one_step_attention(a,h)
    _,s = lstm_cell_post.__call__(context,[c,h])
    c,h = s
    outputs.append(tf.layers.dense
                   (inputs=h,units=len_machine_vocabulary,reuse=tf.AUTO_REUSE,name='outputer'))

outputs = tf.stack(outputs,axis=1)


(?, 64)
(?, 64)


In [16]:
print(outputs.shape)

(?, 10, 11)


In [17]:
print(y_placeholder.shape)

(?, 10, 11)


In [18]:
cost_function = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs,labels=y_placeholder))/9000
print(cost_function.shape)

()


In [19]:
optimizer = tf.train.AdamOptimizer(0.005).minimize(cost_function)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [20]:
ys = Y[0:minibatch_size]
xs = X[0:minibatch_size]
print(ys.shape)
print(sess.run(cost_function,feed_dict={y_placeholder:ys,x_placeholder:xs,now_batch_size:minibatch_size}))
translated_time = tf.argmax(outputs,-1)
standard_time = tf.argmax(y_placeholder,-1)
correct = tf.equal(translated_time,standard_time)
accuracy = tf.reduce_mean(tf.cast(correct,"float"))
print(sess.run(accuracy,feed_dict={y_placeholder:Y,x_placeholder:X,now_batch_size:9000}))

(64, 10, 11)
0.1659565
0.16476667


In [21]:
print(xs.shape)
print(ys.shape)
for e in range(100):
    print('cost:',sess.run(cost_function,feed_dict={y_placeholder:Y,x_placeholder:X,now_batch_size:9000}))
    print('accuracy_train:',sess.run(accuracy,feed_dict={y_placeholder:Y,x_placeholder:X,now_batch_size:9000}))
    print('accuracy_test:',sess.run(accuracy,feed_dict={y_placeholder:Y_test,x_placeholder:X_test,now_batch_size:1000}))
    for i in range(141):
        low = 64*i
        high = 64*(i+1)
        if high>8999:
            high = 8999
        ys = Y[low:high]
        xs = X[low:high]
        sess.run(optimizer,feed_dict={y_placeholder:ys,x_placeholder:xs,now_batch_size:high-low})
    

(64, 30, 37)
(64, 10, 11)
cost: 23.373549
accuracy_train: 0.16476667
accuracy_test: 0.1697
cost: 7.388712
accuracy_train: 0.7244889
accuracy_test: 0.7246
cost: 3.0688174
accuracy_train: 0.88762224
accuracy_test: 0.8791
cost: 1.4145228
accuracy_train: 0.9562889
accuracy_test: 0.9494
cost: 0.7524823
accuracy_train: 0.9785778
accuracy_test: 0.973
cost: 0.40783703
accuracy_train: 0.98928887
accuracy_test: 0.985
cost: 0.2379683
accuracy_train: 0.99432224
accuracy_test: 0.9905
cost: 0.13439785
accuracy_train: 0.9973556
accuracy_test: 0.9947
cost: 0.13980357
accuracy_train: 0.9969222
accuracy_test: 0.9929
cost: 0.10010674
accuracy_train: 0.99754447
accuracy_test: 0.9953
cost: 0.11164998
accuracy_train: 0.9967
accuracy_test: 0.9946
cost: 0.049163014
accuracy_train: 0.9988889
accuracy_test: 0.9971
cost: 0.15397993
accuracy_train: 0.9957111
accuracy_test: 0.9938
cost: 0.34305772
accuracy_train: 0.9901556
accuracy_test: 0.9876
cost: 0.02572367
accuracy_train: 0.99961114
accuracy_test: 0.9982
cost

In [84]:
def predict(s):
    l = []
    for c in s:
        l.append(human_vocab[c])
    one_hot = np.zeros(shape=[1,Tx,len(human_vocab)])
    for i in range(len(s)):
        one_hot[0,i,human_vocab[s[i]]] = 1
    for i in range(len(s),Tx):
        one_hot[0,i,human_vocab['<pad>']] = 1
    print(one_hot.shape)
    pred = sess.run(outputs,feed_dict={x_placeholder:one_hot,now_batch_size:1})
    result = np.argmax(pred,-1)
    print(result)
    print(result.shape)
    print(result[0,9])
    result_s = []
    for i in range(len(result[0])):
        result_s.append(inv_machine_vocab[result[0,i]])
    return result_s

In [103]:
predict(r'6/11/78')

(1, 30, 37)
[[ 2 10  8  9  0  1  7  0  2  2]]
(1, 10)
2


['1', '9', '7', '8', '-', '0', '6', '-', '1', '1']

In [72]:
len(human_vocab)

37

In [43]:
lxn1 = np.array([[1,2,3],[4,5,6]])
lxn1[1][2]

6

In [104]:
saver = tf.train.Saver()


In [105]:
saver.save(sess,r'D:\DeepLearning\model\time_translation')

'D:\\DeepLearning\\model\\time_translation'